In [1]:
import sys, os
from pathlib import Path

sys.path.append(os.path.abspath(Path().resolve().parent/'src'))

In [23]:
from kinematicsrobotics.datahandler import Extract
from kinematicsrobotics.dataprocessing import Preprocessing
from kinematicsrobotics.model import Model
from kinematicsrobotics.metrics import Metrics
from kinematicsrobotics.kinematics import Robo,Spacemapping

## Dataset

In [3]:
ext = Extract()
ext._path_project = os.path.abspath(Path().resolve().parent)

dataset = ext.dataframe( r'src\data\ready\dataset-radius-1cm.csv')

In [4]:
data  = Preprocessing(dataset = dataset, 
                           x_labels=['p_x', 'p_y','p_z', 'roll', 'pich', 'yaw'],
                           y_labels=['theta_1', 'theta_2', 'theta_3', 'theta_4']
                        )

In [5]:
x_train, x_test, y_train, y_test = data.data_train_test

## Modelo

In [6]:
mlp = Model.mlp_regressor(hidden_layer_sizes=(300,300,300,300,300), EPOCHS_NOCHANGE = 20, activation = 'tanh')

## Estimação de Parâmatros

In [19]:
# Estimação de hiperparâmetros
history = ext.dataframe(r'src\data\ready\history.csv')

history_best = history[history['rank_test_score'] == 1]

params = eval(history_best.iloc[0]['params'])

## Treino

In [8]:
mlp.set_model(**params)

In [7]:
mlp.fit(x=x_train,y=y_train)

## Metricas

In [8]:
metrica = Metrics(model = mlp, 
                  preprocessing = data)

In [9]:
metrica.mse(mlp.predict(x =x_train), y_train)

array([0.0007993 , 0.00185118, 0.00466406, 0.00264959])

### Erro juntas

In [11]:
metrica.predict_joint(x = x_train[:1])

array([[1.20199050e+02, 7.06934756e+01, 9.49284651e+01, 4.37578006e-03]])

In [10]:
metrica.mse_joint(x = x_train[:1], y = y_train[:1])

array([3.96207900e-02, 1.70700597e+00, 1.14818702e+00, 1.91474511e-05])

### Erro cartesiano

In [12]:
Elos = [['theta_1',10,0,90,0],
        ['theta_2',0,18,180,0],
        ['theta_3',0,18,-180,0],
        ['theta_4',0,0,90,90],
        ['theta_5',18,0,0,0]
]

robo = Robo("Robo", Elos)

metrica.robotic(robo = robo)

In [68]:
 data._scaler_y.inverse_transform(y_train[:1])

array([[120.,  72.,  96.,   0.]])

In [42]:
a = metrica.predict_joint(x = x_train[:1])
a

array([[1.20199050e+02, 7.06934756e+01, 9.49284651e+01, 4.37578006e-03]])

In [66]:
from numpy import append
_mapping = Spacemapping(robo)
def transfor(array):
            novo_array = []
            for sub_array in array:
                    novo_sub_array = append(sub_array, 0)
                    novo_array.append(novo_sub_array)
            return novo_array
x_real = transfor(x_real)[0]
x_real = [x_real.tolist()]

b = transfor(a)[0]
b = [b.tolist()]
b

[[120.19904971738359,
  70.69347561374522,
  94.92846511120811,
  0.004375780056300016,
  0.0]]

In [60]:
_mapping.operational_space(b, output_format='DataFrame')

,R_11,R_12,R_13,p_x,R_21,R_22,R_23,p_y,R_31,R_32,R_33,p_z
0,-0.354711,-0.503006,-0.788141,-23.229059,-0.206439,0.864283,-0.458691,-13.519119,0.911901,0.0,-0.41041,-21.763765


In [69]:
_mapping.operational_space([[120.,  72.,  96.,   0., 0.]], output_format='DataFrame')

,R_11,R_12,R_13,p_x,R_21,R_22,R_23,p_y,R_31,R_32,R_33,p_z
0,-0.352244,-0.5,-0.791154,-23.66443,-0.203368,0.866025,-0.456773,-13.662665,0.913545,0.0,-0.406737,-21.761536


In [62]:
data._scaler_x.inverse_transform(x_train[:1])

array([[-19.22497119,  33.29862687,  12.47649814,   3.14159265,
         -1.15191731,   2.0943951 ]])

#### t

In [17]:
metrica.predict_operacional(x = x_train[:1])

,p_x,p_y,p_z
0,-23.229059,-13.519119,-21.763765


In [16]:
metrica.mse_operacional(x = x_train[:1])

(array([  16.03271707, 2191.90135246, 1172.39562132]),
 array([[-19.22497119,  33.29862687,  12.47649814]]),
          p_x        p_y        p_z
 0 -23.229059 -13.519119 -21.763765)